<table style="margin: 0; text-align: left; background-color: #050A0E; border: 1px solid lightblue;">
    <tr>
        <td style="direction: rtl; text-align: right; color: #FFF8E7; ">
            <h2 style="color:#FFF8E7;">JobTonic</h2>
            <span style="color:#FFF8E7; font-size: 12px">
                כי חיפוש עבודה ראוי להרגיש כמו התחלה חדשה, לא כמו מסע מייגע.
                במקום להיסחף בגלים של מידע, 
                תנו לנו להיות הרוח שמכוונת את המפרש.
                הכלי
                 מזקק עבורכם את מהות כל משרה — בבהירות, במהירות, ובדיוק שמחזיר לכם שליטה.
                זה לא רק לחפש עבודה. זה לדייק מטרה. זה להתקדם עם ראש צלול, לב פתוח, וביטחון אמיתי.
                בכל שלב בדרך – אנחנו האנרגיה שמזיזה אתכם קדימה.
            </span>
        </td>
<td style="width: 290px; height: 200px; vertical-align: middle;">
            <img src="../JobTonic.png" style="width: 100%; height: 100%; display: block;" />
        </td>
    </tr>
</table>

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from openai import OpenAI

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with Selenium for dynamic websites.
    """

    def __init__(self, url):
        self.url = url

        # Set up Selenium WebDriver with Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # Load the webpage
        driver.get(url)
        time.sleep(5)  # Wait for the page to load completely

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()  # Close the browser

        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
# The system prompt for the careers page
find_careers_system_prompt = "You are provided with a list of links found \
on a webpage. You are able to find the link that is most likely to be a Careers page."
find_careers_system_prompt += "You should respond a single link in the following format: \
    https://another.full.url/careers"

# The system prompt for jobs links
job_link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to a keyword that was provided, \
\n"
job_link_system_prompt += "You should respond in JSON as in this example:"
job_link_system_prompt += """
{
    "links": [
        {"job title": "Software Engineer", "url": "https://full.url/goes/here/software-engineer"},
        {"job title": "Data Scientist", "url": "https://full.url/goes/here/data-scientist"},
        {"job title": "Product Manager", "url": "https://full.url/goes/here/product-manager"},
        {"job title": "Data Analyst", "url": "https://full.url/goes/here/data-analyst"},
        {"job title": "Web Developer", "url": "https://full.url/goes/here/web-developer"}
    ]
}
"""
job_link_system_prompt += "\nIn the case that there are no relevant links, respond with an empty JSON object: {}"

# The system prompt for job postings
job_posting_system_prompt = (
    "You are an assistant specialized in summarizing job postings.\n"
    "Your task is to extract and clearly summarize three things from the input job description:\n"
    "1. How the company describes itself.\n"
    "2. What the day-to-day responsibilities of the role are.\n"
    "3. What the qualifications or requirements are to apply.\n\n"
    "Write the summary in plain, professional English. Keep each section concise (1–2 sentences).\n"
    "Do not copy text verbatim unless necessary. Structure your response under the following headers:\n"
    "- About the Company\n"
    "- Role Responsibilities\n"
    "- Qualifications and Requirements\n"
    "in addition to the following fields:\n"
    "- Company Name\n"
    "- Company URL\n"
    "- Company Location\n"
    "- Company Size\n"
    "- Company Website\n"
    "- Company Founded\n"
    "- Company Mission\n"
    "- Company Values\n"
)
job_posting_system_prompt += "\nYou should respond in JSON as in this example:"
job_posting_system_prompt += """
{
    "company_name": "Tech Innovations Inc.",
    "job_url": "https://full.url/goes/here/software-engineer",
    "company_location": "San Francisco, CA",
    "company_size": "500+ employees",
    "company_website": "https://www.techinnovations.com",
    "company_founded": "2010",
    "company_mission": "To innovate and simplify technology for everyone.",
    "company_values": "Innovation, Integrity, Customer Focus",
    "company_description": "The company is a leading provider of innovative technology solutions.",
    "role_responsibilities": "The role involves developing software applications and collaborating with cross-functional teams.",
    "qualifications_requirements": "Candidates should have a degree in Computer Science and experience with Python."
}
"""
job_posting_system_prompt += "\nIn the case that there are no relevant links, respond with an empty JSON object: {}"

In [5]:
print(find_careers_system_prompt)

You are provided with a list of links found on a webpage. You are able to find the link that is most likely to be a Careers page.You should respond a single link in the following format:     https://another.full.url/careers


In [6]:
print(job_link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to a keyword that was provided, 
You should respond in JSON as in this example:
{
    "links": [
        {"job title": "Software Engineer", "url": "https://full.url/goes/here/software-engineer"},
        {"job title": "Data Scientist", "url": "https://full.url/goes/here/data-scientist"},
        {"job title": "Product Manager", "url": "https://full.url/goes/here/product-manager"},
        {"job title": "Data Analyst", "url": "https://full.url/goes/here/data-analyst"},
        {"job title": "Web Developer", "url": "https://full.url/goes/here/web-developer"}
    ]
}

In the case that there are no relevant links, respond with an empty JSON object: {}


In [9]:
print(job_posting_system_prompt)

You are an assistant specialized in summarizing job postings.
Your task is to extract and clearly summarize three things from the input job description:
1. How the company describes itself.
2. What the day-to-day responsibilities of the role are.
3. What the qualifications or requirements are to apply.

Write the summary in plain, professional English. Keep each section concise (1–2 sentences).
Do not copy text verbatim unless necessary. Structure your response under the following headers:
- About the Company
- Role Responsibilities
- Qualifications and Requirements
in addition to the following fields:
- Company Name
- Company URL
- Company Location
- Company Size
- Company Website
- Company Founded
- Company Mission
- Company Values

You should respond in JSON as in this example:
{
    "company_name": "Tech Innovations Inc.",
    "job_url": "https://full.url/goes/here/software-engineer",
    "company_location": "San Francisco, CA",
    "company_size": "500+ employees",
    "company_we

In [ ]:
# Function

# This function find the career page link of a company 
# based on url
def find_career_page(company_url):
    pass

# This function find all relevant job postings links 
# on the career page and return the relevant 
# job postings links by a keyword
def find_job_postings(career_page_url, keyword):
    pass

# This function scraping each job posting from the list and return a summary of each job and save it to a file
def scrape_job_postings(job_postings):
    pass

# This function wraps the three functions above by taking a company url and a keyword and return a summary of each job posting
def find_job_postings_by_keyword(company_url, keyword):
    career_page_url = find_career_page(company_url)
    job_postings = find_job_postings(career_page_url, keyword)
    scrape_job_postings(job_postings)

In [ ]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)